# OPEN WEATHER ONE CALL API FOR WEATHER VARIABLES
---
---

In [1]:
# https://openweathermap.org/api/one-call-3
# Lesson 5.01 - APIs
# https://note.nkmk.me/en/python-unix-time-datetime/

# Imports
---

In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests

import datetime

%matplotlib inline
pd.set_option('display.max_columns', None)

# Testing: Historical Daily Weather from One Call API
---

In [3]:
url_hist_daily = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658362065&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_daily_req = requests.get(url_hist_daily)
print(f'Status: {hist_daily_req.status_code}')

wd_hist = hist_daily_req.json()
print(wd_hist.keys())

wd_hist_df = json_normalize(wd_hist['data'])
wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_hist_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'data'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,2022-07-20 17:07:45,313.66,311.77,1016,15,282.02,0,0.45,272


In [4]:
wd_hist

{'lat': 38.7735,
 'lon': -121.3637,
 'timezone': 'America/Los_Angeles',
 'timezone_offset': -25200,
 'data': [{'dt': 1658362065,
   'sunrise': 1658321791,
   'sunset': 1658374005,
   'temp': 313.66,
   'feels_like': 311.77,
   'pressure': 1016,
   'humidity': 15,
   'dew_point': 282.02,
   'clouds': 0,
   'wind_speed': 0.45,
   'wind_deg': 272,
   'wind_gust': 0.45,
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}]}]}

In [5]:
url_hist_point = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658363457&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_point_req = requests.get(url_hist_point)
print(f'Status: {hist_point_req.status_code}')

wd_hist = hist_point_req.json()
print(wd_hist.keys())

wd_hist_df = json_normalize(wd_hist['data'])
wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_hist_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'data'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,2022-07-20 17:30:57,313.1,311.1,1016,15,281.58,0,0.89,262


## Function hist_weather(unix_datetime)

In [6]:
def hist_weather(unix_datetime, lat, lon):
    '''
    Using the OpenWeather OneCall 3.0 API to pull historical daily data
    Input: UNIX datetime for day requesting
    Output: Pandas dataframe for that day's weather
    '''
    
    url_hist_point = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_datetime}&appid=d43015e86fe9a04995afe46ef9ad42c4"
    hist_point_req = requests.get(url_hist_point)
    print(f'API Status: {hist_point_req.status_code}')

    wd_hist = hist_point_req.json()

    wd_hist_df = json_normalize(wd_hist['data'])
    wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
           'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

    # Convert dt UNIX to datetime UTC
    wd_hist_df['datetime'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_hist_df

# Testing: Current/7-Day Forecast Daily Weather from One Call API

In [7]:
url_current_daily = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,hourly,alerts&appid=d43015e86fe9a04995afe46ef9ad42c4"
current_daily_req = requests.get(url_current_daily)
print(f'Status: {current_daily_req.status_code}')

wd_current = current_daily_req.json()
print(wd_current.keys())

wd_current_df = json_normalize(wd_current['daily'])
wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# TEMPERATURE - KELVINS
temp_list = []
for i in range(len(wd_current['daily'])):
    avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
    temp_list.append(avg_temp)
wd_current_df['temp_avg'] = temp_list

# FEELS LIKE - KELVINS
fl_list = []
for i in range(len(wd_current['daily'])):
    avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
    fl_list.append(avg_fl)
wd_current_df['feels_like_avg'] = fl_list

# Convert dt UNIX to datetime UTC
wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_current_df.head(10)

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'daily'])


,dt,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,wind_gust,temp_avg,feels_like_avg,datetime
0,1660334400,1014,27,282.39,0,4.10,203,7.13,299.94,298.55,2022-08-12 13:00:00
1,1660420800,1014,13,274.49,0,3.72,203,6.64,300.33,298.70,2022-08-13 13:00:00
2,1660507200,1012,11,273.35,0,3.19,139,5.73,301.60,300.01,2022-08-14 13:00:00
3,1660593600,1007,9,273.66,0,3.25,204,6.28,303.91,302.12,2022-08-15 13:00:00
4,1660680000,1005,8,273.05,4,3.47,196,6.83,305.95,304.00,2022-08-16 13:00:00
5,1660766400,1008,12,279.19,46,3.13,202,5.85,306.14,304.32,2022-08-17 13:00:00
6,1660852800,1007,11,279.12,0,3.42,210,7.74,307.51,305.50,2022-08-18 13:00:00
7,1660939200,1007,11,279.05,1,4.33,212,6.43,306.86,304.87,2022-08-19 13:00:00


## Function curr_fore_weather(lat, lon):

In [8]:
def curr_fore_weather_daily(lat, lon):
    '''
    Using OpenWeather OneCall 3.0 API, takes in the latitude and longitude of a location and returns a dataframe with current + 7-day forecast.
    Input: latitude and longitude of a location
    Output: dataframe
    '''
    
    url_current_daily = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid=d43015e86fe9a04995afe46ef9ad42c4"
    current_daily_req = requests.get(url_current_daily)
    print(f'Status: {current_daily_req.status_code}')

    wd_current = current_daily_req.json()
    print(wd_current.keys())

    wd_current_df = json_normalize(wd_current['daily'])
    wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
           'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

    # TEMPERATURE - KELVINS
    temp_list = []
    for i in range(len(wd_current['daily'])):
        avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
        temp_list.append(avg_temp)
    wd_current_df['temp_avg'] = temp_list

    # FEELS LIKE - KELVINS
    fl_list = []
    for i in range(len(wd_current['daily'])):
        avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
        fl_list.append(avg_fl)
    wd_current_df['feels_like_avg'] = fl_list

    # Convert dt UNIX to datetime UTC
    wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_current_df

# Testing: Hourly Forecast

In [25]:
url_fore_hourly = f"https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,daily,minutely,alerts&appid=d43015e86fe9a04995afe46ef9ad42c4"
fore_hourly_req = requests.get(url_fore_hourly)
print(f'Status: {fore_hourly_req.status_code}')

wd_fore = fore_hourly_req.json()
print(wd_fore.keys())

wd_fore_df = json_normalize(wd_fore['hourly'])
wd_fore_df = wd_fore_df[['dt', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]
wd_fore_df

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'hourly'])


,dt,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,wind_gust
0,1660320000,1015,57,286.31,0,1.62,183,1.93
1,1660323600,1015,52,285.52,0,1.77,217,1.58
2,1660327200,1015,46,285.13,0,2.08,233,1.64
3,1660330800,1015,37,284.13,0,2.29,238,2.09
4,1660334400,1014,26,281.82,0,2.37,232,2.43
5,1660338000,1014,15,276.84,0,2.54,220,2.77
6,1660341600,1013,12,275.00,0,2.62,210,2.98
7,1660345200,1012,11,273.54,0,2.58,203,3.10
8,1660348800,1012,10,272.90,0,2.69,201,3.16
9,1660352400,1011,11,273.42,0,3.42,201,3.25


In [ ]:
def forecast_weather_minutely(lat, lon):
    '''
    Using OpenWeather OneCall 3.0 API, takes in the latitude and longitude of a location and returns a dataframe with current + 7-day forecast.
    Input: latitude and longitude of a location
    Output: dataframe
    '''
    
    url_fore_minutely = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,hourly,daily,alerts&appid=d43015e86fe9a04995afe46ef9ad42c4"
    fore_minutely_req = requests.get(url_fore_minutely)
    print(f'Status: {fore_minutely_req.status_code}')

    wd_fore = fore_minutely_req.json()
    print(wd_fore.keys())

    wd_fore_df = json_normalize(wd_fore['minutely'])
    wd_fore_df = wd_fore_df[['dt', 'pressure', 'humidity', 'dew_point',
           'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

    # TEMPERATURE - KELVINS
    temp_list = []
    for i in range(len(wd_fore['minutely'])):
        avg_temp = round(sum([value for value in wd_fore['minutely'][i]['temp'].values()])/len(wd_fore['minutely'][i]['temp']), 2)
        temp_list.append(avg_temp)
    wd_fore_df['temp_avg'] = temp_list

    # FEELS LIKE - KELVINS
    fl_list = []
    for i in range(len(wd_fore['minutely'])):
        avg_fl = round(sum([value for value in wd_fore['minutely'][i]['feels_like'].values()])/len(wd_fore['minutely'][i]['feels_like']), 2)
        fl_list.append(avg_fl)
    wd_fore_df['feels_like_avg'] = fl_list

    # Convert dt UNIX to datetime UTC
    wd_fore_df['datetime'] = wd_fore_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_fore_df

# Hourly - Need Paid Subscription for Historical Hourly

## Testing: Current/2-Day Forecast Hourly Weather from One Call API
---

In [ ]:
url_current_hourly = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,daily&appid=d43015e86fe9a04995afe46ef9ad42c4"
current_hourly_req = requests.get(url_current_hourly)
print(f'Status: {current_hourly_req.status_code}')

wh_current = current_hourly_req.json()
print(wh_current.keys())

wh_current_df = json_normalize(wh_current['hourly'])
wh_current_df = wh_current_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# Convert dt UNIX to datetime UTC
wh_current_df['datetime'] = wh_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wh_current_df.head()

## Testing: Historical Hourly Weather from History API

In [ ]:
'''
url_hist_hourly = "http://history.openweathermap.org/data/2.5/history/city?lat=38.752125&lon=-121.288010&type=hour&start=1659470400&end=1659470400&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_hourly_req = requests.get(url_hist_hourly)
print(f'Status: {hist_hourly_req.status_code}')

wh_hist = hist_hourly_req.json()
print(wh_hist.keys())

wh_hist_df = json_normalize(wh_hist['data'])
wh_hist_df = wh_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg',
       'wind_gust']]
wh_hist_df.head()
'''